In [1]:
!pip install selenium webdriver-manager

In [17]:
# -*- coding: utf-8 -*-
import os, sys, time, json, socket, tempfile, subprocess, random, re, urllib.parse, urllib.request
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# ==============================================================================
# [설정] 이동해서 크롤링할 카테고리 번호 리스트
# ==============================================================================
CATEGORY_LIST = [
    '194448', # 감자
    '194447', # 고구마
]
# ==============================================================================

# ---------------------------------------------------
# 1) 크롬 실행
# ---------------------------------------------------
CHROME_EXE = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

def pick_free_port(start=9222, end=9350):
    for p in range(start, end+1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(("127.0.0.1", p))
                return p
            except OSError:
                continue
    raise RuntimeError("포트 찾기 실패")

def wait_devtools(port, timeout=12):
    url = f"http://127.0.0.1:{port}/json/version"
    t0 = time.time()
    while time.time() - t0 < timeout:
        try:
            with urllib.request.urlopen(url, timeout=1) as r:
                js = json.loads(r.read().decode("utf-8", "ignore"))
                if js.get("Browser"): return True
        except: time.sleep(0.2)
    return False

tmp_profile = tempfile.mkdtemp(prefix="chrome_profile_")
port = pick_free_port()

args = [
    CHROME_EXE,
    f"--remote-debugging-port={port}",
    f"--user-data-dir={tmp_profile}",
    "--lang=ko-KR",
    "--window-size=1400,950",
    "--disable-http2",
    "--disable-blink-features=AutomationControlled",
    "about:blank"
]
subprocess.Popen(args, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if not wait_devtools(port):
    raise SystemExit("크롬 실행 실패")

print(f"[OK] 크롬 실행됨 (Port: {port})")

# ---------------------------------------------------
# 2) Selenium 연결
# ---------------------------------------------------
opts = Options()
opts.add_experimental_option("debuggerAddress", f"127.0.0.1:{port}")
driver = webdriver.Chrome(options=opts)
wait = WebDriverWait(driver, 20)

# ---------------------------------------------------
# 3) 사용자 준비 대기
# ---------------------------------------------------
print("\n" + "="*70)
print("1. 브라우저가 열리면 '첫 번째 카테고리 페이지'로 직접 이동하세요.")
print("2. 접속이 잘 되었는지 확인하세요.")
print("3. 준비가 완료되면 콘솔에 'y'를 입력하고 엔터를 치세요.")
print("="*70)

ready = input("현재 페이지부터 수집을 시작할까요? (y 입력): ").strip().lower()
if ready != 'y':
    driver.quit()
    sys.exit()

try:
    if not driver.window_handles: raise Exception("탭 없음")
    driver.switch_to.window(driver.window_handles[-1])
except Exception as e:
    print(f"[FATAL] 브라우저 연결 실패: {e}")
    sys.exit()

# ---------------------------------------------------
# 4) JS 스니펫 (전체 스캔)
# ---------------------------------------------------
SCRAPE_JS_ASYNC = r"""
var done = arguments[0];
function sleep(ms){ return new Promise(r=>setTimeout(r, ms)); }
const CARD_SELS = ['li[class^="ProductUnit_productUnit"]', 'li.search-product', 'ul#productList > li'];
const TITLE_SELS = ['div.name', 'div[class*="productName"]'];
const PRICE_SELS = ['strong.price-value', 'strong[class*="priceValue"]'];
const UNIT_SELS  = ['span.unit-price', 'span[class*="unitPrice"]'];
const STAR_SELS  = ['em.rating', 'div[class*="star"]'];
const REV_SELS   = ['span.rating-total-count', 'span[class*="ratingCount"]'];
function intVal(t){ if(!t) return null; const n=(t+'').replace(/[^\d]/g,''); return n?parseInt(n,10):null; }
function floatVal(t){ if(!t) return 0.0; const m=(t+'').replace(/,/g,'').match(/\d+(\.\d+)?/); return m?parseFloat(m[0]):0.0; }
function pick(el, sels){ for(const s of sels){ const x = el.querySelector(s); if(x) return x.textContent.trim(); } return ""; }

async function loadAll(){
  // 최저가를 찾기 위해 1페이지 전체(60개)를 훑음
  let prev = -1;
  for(let i=0; i<12; i++){
    let cards = document.querySelectorAll(CARD_SELS.join(','));
    let now = cards.length;
    if(now > 0 && now === prev) break;
    prev = now;
    window.scrollBy(0, 800);
    await sleep(200);
  }
  return document.querySelectorAll(CARD_SELS.join(','));
}

(async () => {
  try {
    const cards = await loadAll();
    const rows = [];
    cards.forEach(el => {
      let linkEl = el.querySelector('a[href*="/vp/products/"]');
      let link = linkEl ? linkEl.href : "";
      let pid = el.getAttribute('data-product-id') || el.getAttribute('data-item-id') || "";
      if(!pid && link) { let m = link.match(/\/vp\/products\/(\d+)/); if(m) pid = m[1]; }
      const title = pick(el, TITLE_SELS);
      const price = intVal(pick(el, PRICE_SELS));
      const unit_price = intVal(pick(el, UNIT_SELS));
      const rating = floatVal(pick(el, STAR_SELS));
      const review_count = intVal(pick(el, REV_SELS));
      rows.push({product_id: pid, title, price, unit_price, rating, review_count, product_url: link});
    });
    done(rows);
  } catch(e){ done([]); }
})();
"""

# ---------------------------------------------------
# 5) [수정됨] 공통 수집 함수 (최저가 1개만 리턴)
# ---------------------------------------------------
def scrape_current_page(current_cat_id):
    print(f"  데이터 수집 및 최저가 분석 중...")
    try:
        # 1. 일단 다 긁어옴
        rows = driver.execute_async_script(SCRAPE_JS_ASYNC)
        print(f"  -> {len(rows)}개 상품 스캔 완료")

        if not rows:
            return []

        # 2. 가격 정보가 있는 것만 필터링
        valid_items = [r for r in rows if r.get('price') is not None and r['price'] > 0]

        if not valid_items:
            print("  [WARN] 가격 정보가 있는 상품이 없습니다.")
            return []

        # 3. [핵심] 가격 오름차순 정렬 후 가장 싼 1개 선택
        # key=lambda x: x['price'] -> 가격을 기준으로 가장 작은 값을 찾음
        cheapest_item = min(valid_items, key=lambda x: x['price'])

        print(f"  ★ 최저가 선정: {cheapest_item['title']} ({cheapest_item['price']}원)")

        # 메타 정보 추가
        cheapest_item['category_id'] = current_cat_id
        cheapest_item['crawling_dt'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # 리스트로 묶어서 반환 (1개짜리 리스트)
        return [cheapest_item]

    except Exception as e:
        print(f"  [ERROR] 스크립트 실행 실패: {e}")
        return []

# ---------------------------------------------------
# 6) 메인 로직
# ---------------------------------------------------
all_data = []
collected_categories = set()

# --- [Phase 1] 현재 페이지 수집 ---
try:
    current_url = driver.current_url
    print(f"\n[Step 1] 현재 페이지 수집: {current_url}")

    match = re.search(r"/categories/(\d+)", current_url)
    current_cat_id = match.group(1) if match else "manual_start"

    rows = scrape_current_page(current_cat_id)
    all_data.extend(rows)
    collected_categories.add(current_cat_id)
except Exception as e:
    print(f"[ERROR] 초기 페이지 수집 실패: {e}")

# --- [Phase 2] 리스트 순회 (가짜 클릭) ---
for idx, cat_id in enumerate(CATEGORY_LIST):
    if cat_id in collected_categories:
        continue

    sleep_time = random.uniform(5, 8)
    print(f"\n[{idx+1}/{len(CATEGORY_LIST)}] 대기 {sleep_time:.1f}초 후 이동: {cat_id}")
    time.sleep(sleep_time)

    # 판매량순 정렬 & 60개 보기 옵션
    target_url = f"https://www.coupang.com/np/categories/{cat_id}?listSize=60&sorter=saleCountDesc"

    try:
        print(f"  페이지 이동 시도 (링크 클릭 흉내)...")

        js_click_fake = f"""
            var a = document.createElement('a');
            a.href = '{target_url}';
            a.id = 'temp_crawler_link';
            document.body.appendChild(a);
            a.click();
        """
        driver.execute_script(js_click_fake)

        try:
            print("  URL 변경 대기 중...")
            wait.until(EC.url_contains(cat_id))
            print("  -> URL 변경 감지됨")
        except TimeoutException:
            print("  [WARN] 주소창 변경 안됨 (이동 실패?)")

        try:
            print("  상품 목록 로딩 대기 중...")
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li[class^='ProductUnit'], ul#productList")))
            print("  -> 상품 목록 로드됨")
        except TimeoutException:
            print(f"  [WARN] {cat_id} 목록 로딩 타임아웃!")
            continue

            # 수집 실행 (최저가 1개만 반환됨)
        rows = scrape_current_page(cat_id)
        if len(rows) > 0:
            all_data.extend(rows)
            collected_categories.add(cat_id)
        else:
            print("  [WARN] 상품이 0개입니다.")

    except Exception as e:
        print(f"  [ERROR] {cat_id} 이동 중 오류: {e}")
        continue

# ---------------------------------------------------
# 7) 저장
# ---------------------------------------------------
print(f"\n[완료] 총 {len(all_data)}개 데이터 수집됨")

if all_data:
    df = pd.DataFrame(all_data)
    cols = ["category_id", "product_id", "title", "price", "unit_price", "rating", "review_count", "product_url", "crawling_dt"]
    df = df.reindex(columns=[c for c in cols if c in df.columns] + [c for c in df.columns if c not in cols])

    filename = f"coupang_cheapest_{datetime.now().strftime('%H%M%S')}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"파일 저장 완료: {filename}")
else:
    print("수집된 데이터가 없습니다.")

[OK] 크롬 실행됨 (Port: 9222)

1. 브라우저가 열리면 '첫 번째 카테고리 페이지'로 직접 이동하세요.
2. 접속이 잘 되었는지 확인하세요.
3. 준비가 완료되면 콘솔에 'y'를 입력하고 엔터를 치세요.

[Step 1] 현재 페이지 수집: https://www.coupang.com/np/categories/194436?listSize=60&sorter=saleCountDesc
  데이터 수집 및 최저가 분석 중...
  -> 60개 상품 스캔 완료
  ★ 최저가 선정: 곰곰 GAP 콩나물, 300g, 1개 (860원)

[1/2] 대기 6.5초 후 이동: 194448
  페이지 이동 시도 (링크 클릭 흉내)...
  URL 변경 대기 중...
  -> URL 변경 감지됨
  상품 목록 로딩 대기 중...
  -> 상품 목록 로드됨
  데이터 수집 및 최저가 분석 중...
  -> 60개 상품 스캔 완료
  ★ 최저가 선정: 국내산 한입 꿀고구마(햇), 800g, 1개 (3350원)

[2/2] 대기 6.3초 후 이동: 194447
  페이지 이동 시도 (링크 클릭 흉내)...
  URL 변경 대기 중...
  -> URL 변경 감지됨
  상품 목록 로딩 대기 중...
  -> 상품 목록 로드됨
  데이터 수집 및 최저가 분석 중...
  -> 60개 상품 스캔 완료
  ★ 최저가 선정: 한끼 감자(햇), 350g, 1개 (1590원)

[완료] 총 3개 데이터 수집됨
파일 저장 완료: coupang_cheapest_193141.csv
